In [92]:
import pandas as pd
import os
import openai

In [93]:
input_path = "summary_data.csv"
dir_path = os.path.abspath('')
data_path = f"{dir_path}/../data/{input_path}"
data = pd.read_csv(data_path, header=None)

In [94]:
filtered_data = data.drop([3], axis=1)
filtered_data.head()

,0,1,2
0,0,"You enjoy the way smoking calms you, but disli...",1
1,1,You enjoy the feeling of calm and relaxation t...,1
2,2,You enjoy the relief that smoking provides you...,1
3,3,You enjoy the smell of cigarettes but dislike ...,1
4,4,You enjoy the taste of tobacco and are looking...,1


In [95]:
# completion of 0 means the summary was poor, 1 means it satisfied the requirements

filtered_data = filtered_data.rename(columns={1: "prompt", 2: "completion"})
filtered_data.head()

,0,prompt,completion
0,0,"You enjoy the way smoking calms you, but disli...",1
1,1,You enjoy the feeling of calm and relaxation t...,1
2,2,You enjoy the relief that smoking provides you...,1
3,3,You enjoy the smell of cigarettes but dislike ...,1
4,4,You enjoy the taste of tobacco and are looking...,1


In [96]:
#apparently you want 100 samples of each class when finetuning

filtered_data.groupby("completion").count()

,0,prompt
completion,,
0,46,46
1,54,54


In [105]:
filtered_data = filtered_data.astype({"completion": str})

In [106]:
train_data = filtered_data.groupby("completion").sample(frac=0.8, random_state=1).drop([0], axis=1)
val_data = filtered_data.drop(train_data.index).drop([0], axis=1)

In [107]:
train_data.shape

(80, 2)

In [108]:
val_data.shape

(20, 2)

In [109]:
train_data_jsonl = train_data.to_json(orient='records', lines=True)
val_data_jsonl = val_data.to_json(orient='records', lines=True)

In [110]:
with open("train_data.jsonl", "w") as outfile:
    outfile.write(train_data_jsonl)
with open("val_data.jsonl", "w") as outfile:
    outfile.write(val_data_jsonl)

In [111]:
with open(f'{dir_path}/../openai_key.txt') as f:
        openai.api_key = f.read()    

In [112]:
# openai.File.create(file=open("train_data.jsonl", "rb"),purpose='fine-tune')

<File file id=file-kSYGcMWB8v5cvRujjoxYf4x3 at 0x7fe599d079f0> JSON: {
  "bytes": 29731,
  "created_at": 1674343760,
  "filename": "file",
  "id": "file-kSYGcMWB8v5cvRujjoxYf4x3",
  "object": "file",
  "purpose": "fine-tune",
  "status": "uploaded",
  "status_details": null
}

In [113]:
# openai.File.create(file=open("val_data.jsonl", "rb"),purpose='fine-tune')

<File file id=file-cI15giraCnViV40cMJVIwbP0 at 0x7fe58ec69d60> JSON: {
  "bytes": 7425,
  "created_at": 1674343768,
  "filename": "file",
  "id": "file-cI15giraCnViV40cMJVIwbP0",
  "object": "file",
  "purpose": "fine-tune",
  "status": "uploaded",
  "status_details": null
}

In [116]:
# openai.FineTune.create(training_file="file-kSYGcMWB8v5cvRujjoxYf4x3", 
#                        validation_file="file-cI15giraCnViV40cMJVIwbP0", 
#                        model="davinci", compute_classification_metrics=True,
#                        classification_positive_class="1", suffix="summary-classifier")

<FineTune fine-tune id=ft-STBuWrHnWV8iVrkSszXOjOH0 at 0x7fe58ecff860> JSON: {
  "created_at": 1674343916,
  "events": [
    {
      "created_at": 1674343916,
      "level": "info",
      "message": "Created fine-tune: ft-STBuWrHnWV8iVrkSszXOjOH0",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "classification_positive_class": "1",
    "compute_classification_metrics": true,
    "learning_rate_multiplier": null,
    "n_epochs": 4,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-STBuWrHnWV8iVrkSszXOjOH0",
  "model": "davinci",
  "object": "fine-tune",
  "organization_id": "org-dqXnzwpBRx9JeWkSdgAJdRLI",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 29731,
      "created_at": 1674343760,
      "filename": "file",
      "id": "file-kSYGcMWB8v5cvRujjoxYf4x3",
      "object": "file",
      "purpose": "fine-tune",
      "status": "processed",
      "status_details": null
    }
  

In [120]:
openai.FineTune.list()

<OpenAIObject list at 0x7fe58ed3ec20> JSON: {
  "data": [
    {
      "created_at": 1674343916,
      "fine_tuned_model": "davinci:ft-personal:summary-classifier-2023-01-22-02-04-52",
      "hyperparams": {
        "batch_size": 1,
        "classification_positive_class": "1",
        "compute_classification_metrics": true,
        "learning_rate_multiplier": 0.1,
        "n_epochs": 4,
        "prompt_loss_weight": 0.01
      },
      "id": "ft-STBuWrHnWV8iVrkSszXOjOH0",
      "model": "davinci",
      "object": "fine-tune",
      "organization_id": "org-dqXnzwpBRx9JeWkSdgAJdRLI",
      "result_files": [
        {
          "bytes": 17989,
          "created_at": 1674353096,
          "filename": "compiled_results.csv",
          "id": "file-nicqMJ2znGBVzmUUjkIZFB3f",
          "object": "file",
          "purpose": "fine-tune-results",
          "status": "processed",
          "status_details": null
        }
      ],
      "status": "succeeded",
      "training_files": [
        {


In [126]:
res_txt = openai.File.download(id="file-nicqMJ2znGBVzmUUjkIZFB3f").decode("utf-8") 

In [127]:
with open("finetuning_results.txt", "w") as text_file:
    text_file.write(res_txt)

In [130]:
ft_model = "davinci:ft-personal:summary-classifier-2023-01-22-02-04-52"
data = "You want to reduce the number of times you smoke a day, you say you plan to do this by being more motivated."
res = openai.Completion.create(model=ft_model, prompt=data, max_tokens=1)
res['choices'][0]['text']

'1'

## Notes from meeting

Redo the finetuning process with more data, try doubling the dataset. We could try bootstrapping the current classifier to label some of the data to save on time
Bootstrapping - 

In [1]:
ADD PAGE NUMBERS

SyntaxError: invalid syntax (2838068790.py, line 1)